# Keras Sandbox

## Notes and Sources
- Contrastive Loss
- Triplet Loss
https://github.com/maciejkula/triplet_recommendations_keras
- Comparison
https://www.quora.com/What-are-the-advantages-of-using-a-triplet-loss-function-over-a-contrastive-loss-How-would-you-decide-which-to-use

In [16]:
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer

In [13]:
def load_data(config, test_split = 0.2):
    print('Loading data...')
    X_r = np.load(config + "_X_r.npy")
    X_s = np.load(config + "_X_s.npy")
    y = np.load(config + "_y.npy")
    print('Data loaded.')
    return X_r, X_s, y

In [15]:
X_r, X_s, y = load_data("test")

Loading data...
Data loaded.


## Example

In [36]:
from random import seed
from random import randint
from math import ceil
from math import log10

alphabet = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '+', ' ']

In [32]:
# generate lists of random integers and their sum
def random_sum_pairs(n_examples, n_numbers, largest):
    X, y = list(), list()
    for i in range(n_examples):
        in_pattern = [randint(1,largest) for _ in range(n_numbers)]
        out_pattern = sum(in_pattern)
        X.append(in_pattern)
        y.append(out_pattern)
    return X, y

X, y = random_sum_pairs(3, 2, 4)
print(X)
print(y)

[[2, 1], [1, 3], [2, 4]]
[3, 4, 6]


In [33]:
# convert data to strings
def to_string(X, y, n_numbers, largest):
    max_length = n_numbers * ceil(log10(largest+1)) + n_numbers - 1
    Xstr = list()
    for pattern in X:
        strp = '+'.join([str(n) for n in pattern])
        strp = ''.join([' ' for _ in range(max_length-len(strp))]) + strp
        Xstr.append(strp)
    max_length = ceil(log10(n_numbers * (largest+1)))
    ystr = list()
    for pattern in y:
        strp = str(pattern)
        strp = ''.join([' ' for _ in range(max_length-len(strp))]) + strp
        ystr.append(strp)
    return Xstr, ystr

Xstr, ystr = to_string(X, y, 2, 4)
print(Xstr)
print(ystr)

['2+1', '1+3', '2+4']
['3', '4', '6']


In [34]:
# integer encode strings
def integer_encode(X, y, alphabet):
    char_to_int = dict((c, i) for i, c in enumerate(alphabet))
    Xenc = list()
    for pattern in X:
        integer_encoded = [char_to_int[char] for char in pattern]
        Xenc.append(integer_encoded)
    yenc = list()
    for pattern in y:
        integer_encoded = [char_to_int[char] for char in pattern]
        yenc.append(integer_encoded)
    return Xenc, yenc

Xenc, yenc = integer_encode(Xstr, ystr, alphabet)
print(Xenc)
print(yenc)

[[2, 10, 1], [1, 10, 3], [2, 10, 4]]
[[3], [4], [6]]


In [37]:
# one hot encode
def one_hot_encode(X, y, max_int):
    Xenc = list()
    for seq in X:
        pattern = list()
        for index in seq:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        Xenc.append(pattern)
    yenc = list()
    for seq in y:
        pattern = list()
        for index in seq:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        yenc.append(pattern)
    return Xenc, yenc

Xenc, yenc = one_hot_encode(Xenc, yenc, len(alphabet))
print(Xenc)
print(yenc)

[[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]]]
[[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]]]


In [ ]:
seed(1)
n_samples = 1
n_numbers = 2
largest = 10
# generate pairs
X, y = random_sum_pairs(n_samples, n_numbers, largest)
print(X, y)
# convert to strings
X, y = to_string(X, y, n_numbers, largest)
print(X, y)
# integer encode
alphabet = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '+', ' ']
X, y = integer_encode(X, y, alphabet)
print(X, y)
# one hot encode
X, y = one_hot_encode(X, y, len(alphabet))
print(X, y)